In [1]:
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, classification_report
from sklearn.multioutput import MultiOutputClassifier



In [2]:
db = 'refseq_in_house'
region = 'dna_seq' # 'dna_seq' or 'V3V4'. Will only apply to training sequences

In [3]:
print(db)
print(region)

refseq_in_house
dna_seq


In [ ]:
################################ Datas and variables
# paths to reference files
ref_data = '../datasets/train_sets/' + db + '.csv'


# test df import
test_df = pd.read_csv('../datasets/test_sets/test_set_from_refseq_v2.csv')


# samples sequences too classify import
unknown_df = pd.read_csv('../datasets/pre_processed_data/thresh_17/dna_sequences_17.csv')


# levels used for taxonomy
taxonomy_levels = ['domain', 'phylum', 'class', 'order', 'family', 'genus', 'species']





In [109]:
################################ Functions

# sequences segmentation into k-mers, returns list
def generate_kmers(sequence, k):
    """Generate k-mers from a DNA sequence."""
    kmers = []
    for i in range(len(sequence) - k + 1):
        kmers.append(sequence[i:i+k])
    kmers = ' '.join(kmers)
    return kmers

# results evaluation
def print_scores(y_true, y_pred, levels_list, score_file):
    
    for level in levels_list:
        content = ''
        content = content \
            + level \
            + '\n' \
            +  '-' * len(level) \
            + '   accuracy : ' \
            + str(accuracy_score(y_true[level], y_pred[level])) \
            + '   precision :' \
            + str(precision_score(y_true[level], y_pred[level], average = 'weighted', zero_division = np.nan)) \
            + '   score f1 :' \
            + str(f1_score(y_true[level], y_pred[level], average = 'weighted', zero_division = np.nan)) \
            + '\n' \
            + classification_report(y_true[level], y_pred[level], zero_division = np.nan) \
            + '\n\n'

        

        with open(score_file, mode = 'a') as file:
            file.write(content)

        print(content)    
    

# get predictions and probabilities for each sequence
# replace predictions with low probability with 'unclassified'
# returns lists
def pred_and_proba(preds, probas, idx, thresh):
    pred_vec = []
    proba_vec = [max(probas[k][idx]) for k in range(len(taxonomy_levels))]
    for level, proba in enumerate(proba_vec):
        if proba > thresh[level]:
            pred_vec.append(preds[idx][level])
        else:
            pred_vec.extend(["unclassified"] * (len(taxonomy_levels) - level)) # once a level is set to 'unclassified', lower levels are too
            break
        
    return pred_vec, proba_vec


# print 'unclassified' rate for each taxonomy level of a results df
def unclassified_rate(df, score_file):
    for level in taxonomy_levels:
        unclassified_count = df[level].value_counts(normalize = True)
        if 'unclassified' in unclassified_count.index:
            unclassified_rate = unclassified_count['unclassified'] * 100
        else:
            unclassified_rate = 0
        with open(score_file, mode = 'a') as file:
            file.write("Taux de 'unclassified' au rang %s: %.2f%%\n"%(level, unclassified_rate))
        print("Taux de 'unclassified' au rang %s: %.2f%%"%(level, unclassified_rate))
        
    print('\n')











In [110]:
################################ Sequences vectorization

# reference file importation and pre-process
known_df = pd.read_csv(ref_data)
known_df.fillna('undefined', inplace = True)
known_df.drop_duplicates(region, inplace = True)


In [111]:
#######################
# keep only if needed #
#######################

# remove in train_set sequences from test_set
for id in test_df['seq_id']:
    known_df = known_df.loc[(known_df['seq_id'] != id)]

In [112]:

# train_df sequences segmentation
known_df['kmers'] = known_df[region].apply(lambda x: generate_kmers(x, 7))

# test_df sequences segmentation
test_df['kmers'] = test_df['dna_seq'].apply(lambda x: generate_kmers(x, 7))

# train and test data
X_train = known_df['kmers']
y_train = known_df[taxonomy_levels]

X_test = test_df['kmers']
y_test = test_df[taxonomy_levels]

# Features vectorization
vectorizer = CountVectorizer()

X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
################################ Model training

# params for cross validation
params = {'estimator__alpha' : [1, 0.1, 0.05, 0.01, 0.005, 0.001]}

# model
model = MultinomialNB()

moc_cv_model = GridSearchCV(MultiOutputClassifier(model), param_grid = params)

moc_cv_model.fit(X_train_vectorized, y_train)

# model saving
joblib.dump(moc_cv_model, '../models/NB_' + db + '_' + region + '.joblib')

        

/home/marthe/miniconda3/envs/microbiote/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:302: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  self.y_type_ = type_of_target(y, input_name="y")
/home/marthe/miniconda3/envs/microbiote/lib/python3.11/site-packages/sklearn/utils/multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  ys_types = set(type_of_target(x) for x in ys)
/home/marthe/miniconda3/envs/microbiote/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:359: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  y_is_multilabel = type_of_target(y).startswith("multilabel")
/home/marthe/miniconda3/envs/microbiote/lib/p

['/home/marthe/Documents/DS/projet/local/frogsdays/models/NB_refseq_in_house_dna_seq.joblib']

In [ ]:
################################ Model prediction

moc_cv_model = joblib.load('../models/NB_' + db + '_' + region + '.joblib')

print('DONNÉES DE RÉFÉRENCE : ', db)
print('Restriction des séquences : ', region)
print('----------------------------------------------\n\n')


# predictions for test_df
y_pred_moc_cv = moc_cv_model.predict(X_test_vectorized)
y_pred_moc_cv_proba = moc_cv_model.predict_proba(X_test_vectorized)

# results agregation in dataframe, with set thresholds to get rid of low probability predictions
proba_columns = ['proba_'+level for level in taxonomy_levels]
pred_results = pd.DataFrame(columns = taxonomy_levels + proba_columns, index = X_test.index)

thresh = [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7]

for k in range(pred_results.shape[0]):
    pred_vec_k, proba_vec_k = pred_and_proba(y_pred_moc_cv, y_pred_moc_cv_proba, k, thresh)
    pred_results.loc[k] = pred_vec_k + proba_vec_k

# test predictions saving
pred_results_to_save = pd.concat([test_df['seq_id'], pred_results], axis = 1)
pred_results_to_save.to_csv('../preds/NB/NB_test_' + db + '_' + region + '.csv', index = False)


score_file = '../scores/NB/scoresNB_'+ db + '_' + region + '.txt'

# test predictions evaluation
with open(score_file, mode = 'a') as file:
    file.write('DONNÉES DE RÉFÉRENCE : ' + db + '\n')
    file.write('Restriction des séquences : ' + region + '\n')
    file.write('----------------------------------------------\n\n')
unclassified_rate(pred_results, score_file)
print_scores(y_test, pred_results, taxonomy_levels, score_file)




DONNÉES DE RÉFÉRENCE :  refseq_in_house
Restriction des séquences :  dna_seq
----------------------------------------------


Taux de 'unclassified' au rang domain: 0.00%
Taux de 'unclassified' au rang phylum: 0.00%
Taux de 'unclassified' au rang class: 0.00%
Taux de 'unclassified' au rang order: 0.00%
Taux de 'unclassified' au rang family: 0.00%
Taux de 'unclassified' au rang genus: 0.00%
Taux de 'unclassified' au rang species: 4.51%


domain
------   accuracy : 1.0   precision :1.0   score f1 :1.0
              precision    recall  f1-score   support

    Bacteria       1.00      1.00      1.00       843

    accuracy                           1.00       843
   macro avg       1.00      1.00      1.00       843
weighted avg       1.00      1.00      1.00       843



phylum
------   accuracy : 0.99644128113879   precision :0.996482822604096   score f1 :0.9964358874810249
                         precision    recall  f1-score   support

         Actinomycetota       1.00      0.98    

In [115]:
# # ############################### Samples sequences classification
# # sequences segmentation
# unknown_df['kmers'] = unknown_df['dna_seq'].apply(lambda x: generate_kmers(x, 7))

# # features vectorization
# X_unknown = unknown_df['kmers']
# X_unknown_vectorized = vectorizer.transform(X_unknown)

# # prediction
# samples_prediction = moc_cv_model.predict(X_unknown_vectorized)
# samples_proba = moc_cv_model.predict_proba(X_unknown_vectorized)

# # results agregation in dataframe, with set thresholds to get rid of low probability predictions
# proba_columns = ['proba_'+level for level in taxonomy_levels]
# samples_results = pd.DataFrame(columns = ['seq_id'] + taxonomy_levels + proba_columns, index = X_test.index)

# thresh = [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7]

# for k in range(samples_prediction.shape[0]):
#     seq_id = [unknown_df['seq_id'][k]]
#     pred_vec_k, proba_vec_k = pred_and_proba(samples_prediction, samples_proba, k, thresh)
#     item = seq_id + pred_vec_k + proba_vec_k
#     samples_results.loc[k] = item

# # samples prediction saving
# samples_results.to_csv('/home/marthe/Documents/DS/projet/local/frogsdays/preds/samples/NB_samples_' + db + '_' + region + '.csv', index = False)



In [116]:
preds = pd.DataFrame(y_pred_moc_cv, columns = taxonomy_levels, index = X_test.index)
preds

,domain,phylum,class,order,family,genus,species
0,Bacteria,Pseudomonadota,Gammaproteobacteria,Alteromonadales,Shewanellaceae,Shewanella,Shewanella putrefaciens
1,Bacteria,Bacillota,Clostridia,Eubacteriales,Oscillospiraceae,Faecalibacterium,Faecalibacterium prausnitzii
2,Bacteria,Actinomycetota,Actinomycetes,Micrococcales,Micrococcaceae,Pseudoglutamicibacter,Pseudoglutamicibacter albus
3,Bacteria,Pseudomonadota,Betaproteobacteria,Burkholderiales,Comamonadaceae,Comamonas,Comamonas thiooxydans
4,Bacteria,Bacillota,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus pneumoniae
...,...,...,...,...,...,...,...
838,Bacteria,Actinomycetota,Actinomycetes,Micrococcales,Micrococcaceae,Paeniglutamicibacter,Glutamicibacter mishrai
839,Bacteria,Actinomycetota,Actinomycetes,Actinomycetales,Actinomycetaceae,Arcanobacterium,Arcanobacterium urinimassiliense
840,Bacteria,Actinomycetota,Actinomycetes,Micrococcales,Microbacteriaceae,Microbacterium,Microbacterium ginsengiterrae
841,Bacteria,Pseudomonadota,Betaproteobacteria,Burkholderiales,Burkholderiaceae,Cupriavidus,Cupriavidus plantarum


In [ ]:
score_file = '../scores/recap_scores.csv'

with open(score_file, mode = 'a') as file:
    for level in taxonomy_levels[-3:]:
        true_labels = y_test[level]
        pred_labels = preds[level]
        print(level)
        accuracy = accuracy_score(true_labels, pred_labels)
        print(accuracy)
        precision = precision_score(true_labels, pred_labels, average = 'weighted', zero_division = np.nan)
        print(precision)
        f1 = f1_score(true_labels, pred_labels, average = 'weighted', zero_division = np.nan)
        print(f1)
        
        file.write('NB, {}, {}, {}, {}, {}, {}\n'.format(db, region, level, accuracy, precision, f1))


family
0.9881376037959668
0.989574143196471
0.9875492831649438
genus
0.9134045077105575
0.9641075086751119
0.9102209856313448
species
0.5599051008303677
0.9018479033404406
0.5506044107467595
